In [37]:
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np
train = pd.read_csv('kc_house_data.csv') 
corr_matrix = train.corr()
corr_matrix["price"].sort_values(ascending=False)

price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
floors           0.256794
yr_renovated     0.126434
sqft_lot         0.089661
sqft_lot15       0.082447
yr_built         0.054012
condition        0.036362
long             0.021626
id              -0.016762
zipcode         -0.053203
Name: price, dtype: float64

In [38]:
cols = ['price','sqft_living','grade','sqft_above','sqft_living15','bathrooms','view','sqft_basement','bedrooms', 'lat', 'waterfront', 'floors','yr_renovated']
train = train[cols]

In [39]:
train = pd.get_dummies(train)
train = train.fillna(train.mean())
train.shape

(21613, 13)

In [40]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train = train[['sqft_living','grade','sqft_above','sqft_living15','bathrooms','view','sqft_basement','bedrooms', 'lat', 'waterfront', 'floors', 'yr_renovated']]
X_train = scale.fit_transform(X_train)
y_train = train['price'].values
seed = 7
np.random.seed(seed)

In [41]:
def build_model():
    # create model
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(keras.layers.Dense(30, activation='relu'))
    model.add(keras.layers.Dense(40, activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    model.compile(optimizer ='adam', loss = 'mean_squared_error', metrics =["mse"])
    return model

In [42]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    X_val = X_train[i * num_val: (i + 1) * num_val]
    Y_val = y_train[i * num_val: (i + 1) * num_val]

    # Prepare the training data: data from all other partitions
    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    Y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(X_train_part, Y_train_part,epochs=num_epochs, verbose=0)
    # Evaluate the model on the validation data 
    val_loss, val_acc = model.evaluate(X_val, Y_val, verbose=0)
    all_scores.append(val_acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [44]:
np.mean(all_scores)

38878933000.0

In [3]:
import numpy as np
np.sqrt(38878933000.0)

197177.4150352925